In [ ]:
-- Streaming: Flink SQL
CREATE TABLE Bid (
    `auction` BIGINT, 
    `bidder` BIGINT, 
    `price` DOUBLE, 
    `datetime` TIMESTAMP(3),
    WATERMARK FOR datetime AS `datetime` - INTERVAL '1' SECOND
) WITH ('connector' = 'datagen');

EXPLAIN PLAN FOR 
SELECT window_start, window_end, AVG(price) AS avg_price
FROM TABLE(HOP(TABLE Bid, DESCRIPTOR(datetime), INTERVAL '2' SECONDS, INTERVAL '10' SECONDS))
GROUP BY window_start, window_end;


| == Abstract Syntax Tree ==
LogicalAggregate(group=[{0, 1}], avg_price=[AVG($2)])
+- LogicalProject(window_start=[$4], window_end=[$5], price=[$2])
   +- LogicalTableFunctionScan(invocation=[HOP(DESCRIPTOR($3), 2000:INTERVAL SECOND, 10000:INTERVAL SECOND)], rowType=[RecordType(BIGINT auction, BIGINT bidder, DOUBLE price, TIMESTAMP(3) *ROWTIME* datetime, TIMESTAMP(3) window_start, TIMESTAMP(3) window_end, TIMESTAMP(3) *ROWTIME* window_time)])
      +- LogicalProject(auction=[$0], bidder=[$1], price=[$2], datetime=[$3])
         +- LogicalWatermarkAssigner(rowtime=[datetime], watermark=[-($3, 1000:INTERVAL SECOND)])
            +- LogicalTableScan(table=[[default_catalog, default_database, Bid]])

== Optimized Physical Plan ==
Calc(select=[window_start, window_end, avg_price])
+- GlobalWindowAggregate(window=[HOP(slice_end=[$slice_end], size=[10 s], slide=[2 s])], select=[AVG((sum$0, count$1)) AS avg_price, COUNT(count1$2) AS window_start, start('w$) AS window_end])
   +- Exchange(distribution=[single])
      +- LocalWindowAggregate(window=[HOP(time_col=[datetime], size=[10 s], slide=[2 s])], select=[AVG(price) AS (sum$0, count$1), COUNT(*) AS count1$2, slice_end('w$) AS $slice_end])
         +- Calc(select=[price, datetime])
            +- WatermarkAssigner(rowtime=[datetime], watermark=[-(datetime, 1000:INTERVAL SECOND)])
               +- TableSourceScan(table=[[default_catalog, default_database, Bid]], fields=[auction, bidder, price, datetime])

== Optimized Execution Plan ==
Calc(select=[window_start, window_end, avg_price])
+- GlobalWindowAggregate(window=[HOP(slice_end=[$slice_end], size=[10 s], slide=[2 s])], select=[AVG((sum$0, count$1)) AS avg_price, COUNT(count1$2) AS window_start, start('w$) AS window_end])
   +- Exchange(distribution=[single])
      +- LocalWindowAggregate(window=[HOP(time_col=[datetime], size=[10 s], slide=[2 s])], select=[AVG(price) AS (sum$0, count$1), COUNT(*) AS count1$2, slice_end('w$) AS $slice_end])
         +- Calc(select=[price, datetime])
            +- WatermarkAssigner(rowtime=[datetime], watermark=[(datetime - 1000:INTERVAL SECOND)])
               +- TableSourceScan(table=[[default_catalog, default_database, Bid]], fields=[auction, bidder, price, datetime])
 |